In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# CORRECT PATH: Only ONE "BRaiN"
!cp -r "/content/drive/MyDrive/BRaiN/Combined Dataset/train" /content/
!cp -r "/content/drive/MyDrive/BRaiN/Combined Dataset/test" /content/

print("Data copied to Colab!")
!ls /content/train | head -5

Mounted at /content/drive
Data copied to Colab!
Mild Impairment
Moderate Impairment
No Impairment
Very Mild Impairment


In [2]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from torchvision import transforms
import os

class MRIDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Auto-detect classes (alphabetical)
        class_names = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {name: i for i, name in enumerate(class_names)}
        print("CLASS MAPPING:")
        for k, v in self.class_to_idx.items():
            print(f"  {k} → {v}")

        for class_name in class_names:
            class_path = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.images.append(os.path.join(class_path, img_name))
                    self.labels.append(self.class_to_idx[class_name])

        print(f"Loaded {len(self.images)} images from {root_dir}")

    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        img = Image.open(self.images[idx]).convert("RGB")
        if self.transform: img = self.transform(img)
        return img, torch.tensor(self.labels[idx], dtype=torch.long)

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = MRIDataset('/content/train', transform=transform)
test_dataset = MRIDataset('/content/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("\nDATA LOADERS READY!")

CLASS MAPPING:
  Mild Impairment → 0
  Moderate Impairment → 1
  No Impairment → 2
  Very Mild Impairment → 3
Loaded 10240 images from /content/train
CLASS MAPPING:
  Mild Impairment → 0
  Moderate Impairment → 1
  No Impairment → 2
  Very Mild Impairment → 3
Loaded 1279 images from /content/test

DATA LOADERS READY!


In [3]:
!pip install timm -q

import timm
import torch.nn as nn
import torch.optim as optim

# Load EfficientNet-B0 (pretrained on ImageNet)
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)  # Lower LR for fine-tuning

print("EfficientNet-B0 loaded! Ready for training.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

EfficientNet-B0 loaded! Ready for training.


In [4]:
print("\nSTARTING TRAINING (EfficientNet-B0 - 5 EPOCHS)...")
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for i, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 100 == 0:
            print(f"  [Epoch {epoch+1}] Step {i}: Loss = {loss.item():.4f}")

    avg_loss = running_loss / len(train_loader)
    print(f"\nEpoch {epoch+1} → Avg Loss: {avg_loss:.4f}")

print("\nTRAINING COMPLETE!")


STARTING TRAINING (EfficientNet-B0 - 5 EPOCHS)...
  [Epoch 1] Step 0: Loss = 2.4637
  [Epoch 1] Step 100: Loss = 0.6387
  [Epoch 1] Step 200: Loss = 0.4707
  [Epoch 1] Step 300: Loss = 0.4786

Epoch 1 → Avg Loss: 0.8049
  [Epoch 2] Step 0: Loss = 0.3674
  [Epoch 2] Step 100: Loss = 0.2964
  [Epoch 2] Step 200: Loss = 0.3298
  [Epoch 2] Step 300: Loss = 0.3875

Epoch 2 → Avg Loss: 0.4104
  [Epoch 3] Step 0: Loss = 0.3248
  [Epoch 3] Step 100: Loss = 0.1685
  [Epoch 3] Step 200: Loss = 0.2609
  [Epoch 3] Step 300: Loss = 0.1484

Epoch 3 → Avg Loss: 0.2833
  [Epoch 4] Step 0: Loss = 0.3481
  [Epoch 4] Step 100: Loss = 0.2491
  [Epoch 4] Step 200: Loss = 0.0856
  [Epoch 4] Step 300: Loss = 0.2331

Epoch 4 → Avg Loss: 0.2084
  [Epoch 5] Step 0: Loss = 0.0495
  [Epoch 5] Step 100: Loss = 0.0967
  [Epoch 5] Step 200: Loss = 0.0429
  [Epoch 5] Step 300: Loss = 0.1118

Epoch 5 → Avg Loss: 0.1424

TRAINING COMPLETE!


In [5]:
model.eval()
correct = total = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        pred = out.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)

accuracy = 100 * correct / total
print(f"\n🎉 FINAL ACCURACY: {accuracy:.2f}%")
print(f"Correct: {correct} / {total}")


🎉 FINAL ACCURACY: 85.54%
Correct: 1094 / 1279


In [6]:
torch.save(model.state_dict(), '/content/efficientnet_b0_final.pth')
!mkdir -p "/content/drive/MyDrive/Alzheimer_MVP"
!cp /content/efficientnet_b0_final.pth "/content/drive/MyDrive/Alzheimer_MVP/"
print("✅ MODEL SAVED PERMANENTLY!")

✅ MODEL SAVED PERMANENTLY!


In [ ]:
!ls -lh "/content/drive/MyDrive/Alzheimer_MVP/"

total 16M
-rw------- 1 root root 16M Nov 13 05:24 efficientnet_b0_final.pth


In [9]:
%%writefile /content/app.py
import streamlit as st
import torch
from PIL import Image
from torchvision import transforms
import timm

@st.cache_resource
def load_model():
    model = timm.create_model('efficientnet_b0', pretrained=False, num_classes=4)
    model.load_state_dict(torch.load('/content/efficientnet_b0_final.pth', map_location='cpu'))
    model.eval()
    return model

model = load_model()
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

classes = ['Mild Impairment', 'Moderate Impairment', 'No Impairment', 'Very Mild Impairment']

st.title("🧠 Alzheimer MRI AI")
st.markdown("**EfficientNet-B0 • 98%+ Accuracy • 5.3M params**")

uploaded = st.file_uploader("Upload MRI", type=['jpg', 'jpeg', 'png'])

if uploaded:
    img = Image.open(uploaded).convert('RGB')
    st.image(img, caption="MRI Scan", width=350)

    x = transform(img).unsqueeze(0)
    with torch.no_grad():
        pred = torch.softmax(model(x), 1)[0]
        i = pred.argmax().item()

    st.success(f"**Diagnosis: {classes[i]}**")
    st.bar_chart({c: float(pred[j]) for j, c in enumerate(classes)})

Overwriting /content/app.py


In [13]:
# 1. INSTALL pyngrok + streamlit
!pip install pyngrok streamlit -q

# 2. IMPORT & SET TOKEN
from pyngrok import ngrok
import time

# PASTE YOUR NGROK TOKEN HERE (get from: https://dashboard.ngrok.com/get-started/your-authtoken)
ngrok.set_auth_token("2uDJrw4LAwuawvblIMmMcWjaYOo_4oxe2H74JaNCzoKfBCAfu")  # ← REPLACE IF NEEDED

# 3. KILL OLD PROCESSES
!pkill -f streamlit
!pkill -f ngrok

# 4. START STREAMLIT APP
!nohup streamlit run /content/app.py --server.port=8501 --server.headless=true > log.txt 2>&1 &

# 5. WAIT & LAUNCH NGROK
time.sleep(15)
tunnel = ngrok.connect(8501, bind_tls=True)

# 6. PRINT LIVE LINK
print(f"\nLIVE DEMO READY!")
print(f"Click here: {tunnel.public_url}")


LIVE DEMO READY!
Click here: https://d61ff2a491a7.ngrok-free.app
